# Dynamic Mapping

In many cases, data for creating maps are retrievable in regular intervals or available historically. In such cases, 
dynamic mapping can uncover additional insights beyond what static maps can offer.  

Dynamic mapping, however, is more challenging than simply creating a (static) market map at each point in time (see [[1]](#ref_1)).
The dynamic mapping framework *EvoMap* provides one solution to this problem. *EvoMap* integrates various static mapping methods 
and allows the analyst to use them for creating dynamic market maps. Thereby, it provides the analyst with a high degree of control 
over its output (e.g., by letting the analyst set certain constraints on the resultant maps, such as the degree of smoothing). 

For methodological background and a detailed description, see the original paper [[1]](#ref_1).

This tutorial demonstrates how to use *EvoMap* to create dynamic market maps from time-evolving relationship data. The tutorial covers data preparation, running the method, evaluating its results, tuning its hyperparameter and several useful functions provided as part of this package (e.g., for drawing dynamic market maps).  

## Sections

1. [Data Preparation](#section_1)

2. [Running the Method](#section_2)

3. [Output Exploration](#section_3)

4. [Quantitative Evaluation](#section_4)

5. [Hyperparameter Tuning](#section_5)

6. [Chosing Different Mapping Methods](#section_6)

7. [Special Cases and Extensions](#section_7)

## Background

**Author:** Maximlian Matthe (matthe@wiwi.uni-frankfurt.de)

**Last updated:** May 2022

*If you use this package or parts of its code, please cite our work!*

**References**

<a id = "ref_1">[1]</a> Matthe, M., Ringel, D. M., Skiera, B. (2022), "Mapping Market Structure Evolution", Working Paper.

## Data Preparation
<a id = 'section_1'></a>
First, load all required imports for this tutorial and set the seed to ensure reproducibility. 

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

np.random.seed(123) 

For this tutorial, we use a subsample of the 'Text-Based Network Industry Classification' data also used in [[1]](#ref_1). The original data is provided at https://hobergphillips.tuck.dartmouth.edu/. The data consists of a time-indexed edgelist, that is, firm-firm relationships. 

For more background on TNIC data, see [[3]](#ref_2). If you intend to use these data, make sure to cite these authors' original work!

<a id = "ref_2">[2]</a> Hoberg, G & Phillips, G. (2016), "Text-Based Network Industries and Endogenous Product Differentiation.", Journal of Political Economy 124 (5), 1423-1465.

We augment the TNIC edgelist with further firm information. Moreover, we limit our analysis to a subsample of firms (i.e., those which are present throughout the whole observation period of 20 years). Our augmented data includes the following variables: 

| Variable | Description | Type |
| --- | --- | --- |
| year | Fiscal year | int |
| gvkey | Firm identifier | int |
| score | Similarity score | float |
| name | Firm name | string |
| size | Firm size (synthetic variable, correlated with market value) | float |
| sic | SIC Code | string |

In the resultant dataset, each row corresponds to a single firm-firm pair at a specific point in time. Thus, each firm variable appears twice in each row (once for each firm). We provide this dataset in the `evomaps` module.

In [ ]:
from evomap.datasets import load_tnic_sample_small
df_sample = load_tnic_sample_small()
df_sample.head()

The original sample includes more than 1,000 different firms. To simplify our demonstration (and accelerate the runtime of this notebook), we pick a smaller subsample from these data focused on the biggest firms (by market value) and their competitors. For details on how we generated this subsample, see <a href = 'TNIC subsample construction.ipynb'>here</a>.

In [ ]:
print("The subsample includes {0} firms and {1} years".format(df_sample.name1.nunique(), df_sample.year.nunique()))

We need to turn this edgelist into a sequence of relationship matrices. Specifically, the expected input for *EvoMap* is a list of numpy ndarrays, each containing the pairwise relationships among all firms at a specific point in time. 

**Important:** Make sure that each matrix has the same size and that its rows & columns are ordered consistently! If you do not observe certain firms at some points in time (e.g., due to entry or exit), you should provide inclusion vectors (covered [later](#inclusions)).

`evomap.preprocessing` provides helpful functions for the required transformations: 

In [ ]:
from evomap.preprocessing import edgelist2matrix, sim2dist

def build_distance_matrices(df):
    """Build a sequence of distance matrices from the tnic edgelist.

    As the tnic data consists of pairwise similarities, we apply the 'sim2dist' 
    function to each similarity matrix to yield distance matrices.

    Parameters
    ----------
    df : pd.DataFrame
        Tnic data in edgelist format. 

    Returns
    -------
    Ds, names, years
        Ds: list of distance matrices, each as a numpy ndarray
        names: array of firm labels
        years: array o year labels
    """
    years = df['year'].sort_values().unique()
    Ds = []
    for year in years:
        S_t, names = edgelist2matrix(
            df = df,
            score_var = 'score',
            id_var_i = 'name1',
            id_var_j = 'name2',
            time_var = 'year',
            time_selected = year)
        D_t = sim2dist(S_t, transformation = 'inverse')
        Ds.append(D_t)
                
    return Ds, names, years

Ds, names, years = build_distance_matrices(df_sample)
n_samples = Ds[0].shape[0]
n_periods = len(Ds)

## Running EvoMap
<a id = 'section_2'></a>

EvoMap is part of the `evomap.mapping` module. 

Before we run the model, we first fix a random starting initialization. While one does not need to provide a starting initialization explicitly, doing so is helpful when comparing the derived maps to solutions by other approaches. 

In [ ]:
Y_init = np.random.normal(0.0,1.0,size=(n_samples,2))
Y_inits = [Y_init]*n_periods

Then, choose a value for the hyperparamter alpha, initialize the model accordingly, and fit it to the data. We will cover ways how to identify appropriate values for EvoMap's hyperparameters [later](#section_5)

In [ ]:
from evomap.mapping.evomap import EvoTSNE
model = EvoTSNE(
    verbose = 2,
    alpha = 0.001)

Ys = model.fit_transform(Ds)

The output is a list of map coordinates, each stored in a ndarray of shape (n_samples, d), where d is typically = 2. 

This package provides multiple tools to explore these results. Here, we look at the first 4 maps as a sequence. We cover more advanced alternatives for exploration in [Section 3: Exploring Model Output](#section_3).

In [ ]:
from evomap.printer import draw_map_sequence
draw_map_sequence(Ys[:4])

<a id = 'section_3'></a>
## Exploring Model Output

This package provides three ways to explore EvoMap's output: 

1. [Draw a static map](#static_map)
2. [Draw a sequence of static maps](#sequence)
3. [Draw a dynamic map](#dynamic_map) (i.e., an overlay of subsequent maps)

All necessary functions are included in the "printer" module and provide lots of flexibility to adjust their aesthetics.

<a id = 'static_map'></a>
### Draw a Static Map

To draw a single static map, simply use the corresponding function ```draw_map()```.

In [ ]:
from evomap.printer import draw_map

draw_map(Ys[0])

Aesthetics of the map can easily be adjusted via additional arguments. For instance, if class labels are available (e.g., obtained via clustering or additional metadata), they can be added as colors. Here, we can use SIC codes for coloring:

In [ ]:
sic_codes = pd.DataFrame({'name1' : names}).merge(df_sample[['name1', 'sic1']], on = 'name1', how = 'left').drop_duplicates('name1')['sic1'].map(lambda x: str(x)[:1]).values
draw_map(Ys[0], c = sic_codes)

In [ ]:
sizes = pd.DataFrame({'name1' : names}).merge(df_sample[['name1', 'size1']], on = 'name1', how = 'left').drop_duplicates('name1')['size1'].values
draw_map(Ys[0], c = sic_codes, size = sizes)

One can further annotate the map, using clusters

In [ ]:
draw_map(Ys[0], c = sic_codes, size = sizes, annotate = 'clusters')

or labels. Note that one can also use additional keyword arguments to adjust the plot and its labels further. 

In [ ]:
draw_map(
    Ys[0], 
    c = sic_codes, 
    size = sizes,
    annotate = 'labels', 
    labels = names, 
    highlight_labels = ['APPLE INC', 'ADOBE INC'],
    fig_size = (10,10),
    fontdict = {'size': 8},
    scatter_kws = {'s' : 20})

<a id = 'sequence'></a>
### Draw a Sequence of Maps

Rather than drawing a single map, one can also draw a sequence via ```draw_map_sequence()```.

The function takes a list of ndarrays (each containing the map coordinates for one period) and creates a map for each of them:

In [ ]:
from evomap.printer import draw_map_sequence
draw_map_sequence(Ys[:4], n_cols = 4, time_labels = years)

Map aesthetics can be adjusted analogously to drawing a static map. To do so, simply provide arguments of ```draw_map()``` as keyword arguments to ```draw_map_sequence()```:

In [ ]:
draw_map_sequence(Ys[:4], time_labels = years, 
                  c = sic_codes, labels = names, highlight_labels = ['APPLE INC', 'INTUIT INC'])

<a id = 'dynamic_map'></a>
### Draw a Dynamic Map

The third option - and often the most interesting one - is to explore all periods jointly via a *dynamic map* (i.e., an overlay of multiple subsequent maps). To do so, use ```draw_dynamic_map()``` as follows:

In [ ]:
from evomap.printer import draw_dynamic_map

draw_dynamic_map(Ys)

```draw_dynamic_map()``` also provides some options to reveal the individual trajectories of each firm. 

Thereby, all arguments for ```draw_map()``` can also be passed to ```draw_dynamic_map()``` as keyword arguments to control the resultant map's aesthetics. Naturally, the arguments of both functions can also be combined arbitrarily (for instance, highlighting labels for same objects, while highlighting the trajectories of others): 

In [ ]:
draw_dynamic_map(Ys, show_arrows = True, show_last_positions_only= True, highlight_trajectories = ['WALMART INC', 'AT&T INC'],
                 labels = names, highlight_labels = ['PFIZER INC', 'MORGAN STANLEY'])

Besides drawing the full map, one can also focus on individual firms and their trajectories. To do so, the dedicated function 

```draw_trajectories()``` 

is available. 

Besides map coordinates and labels (e.g., firm names), this function expects you to provide a list (or array) of focus firms, for which the trajectories should be displayed:

In [ ]:
from evomap.printer import draw_trajectories
focus_firms = [
    'INTUIT INC',
    'WESTERN DIGITAL CORP', 
    'APPLE INC']    
draw_trajectories(Ys, labels = names, selected_labels = focus_firms)

To ease interpretation, annotations for all periods can be added explicitly:

In [ ]:
draw_trajectories(Ys, labels = names, selected_labels = focus_firms, title_str = "Selected Trajectories", period_labels = years)

<a id = 'section_4'></a>
## Evaluating Model Output

How "good" are these maps (e.g., how well do they fit the input data? How well do they reveal underyling changes?)? 

To answer this question, the following metrics are available. All functions are located within the 'metrics' module.


| Metric | Description | Function | Computed for | 
| --- | --- | --- | --- |
| Hitrate | Nearest neighbor recovery (in %) | ```hitrate_score()``` | Single Map| 
| Adjusted Hitrate | Hitrate, adjusted for random agreement | ```adjusted_hitrate_score()``` | Single Map |
| Avg. Hitrate | Avgerage Hitrate | ```avg_adjusted_hitrate_score()``` | Sequence of Maps |
| Avg. Adjusted Hitrate | Adjusted Hitrate, averaged across subsequent maps | ```avg_adjusted_hitrate_score()``` | Sequence of Maps |
| Misalignment | Average Distance of subsequent map positions  | ```misalign_score()```  | Sequence of Maps |
| Alignment | Cosine similarity of subsequent map positions | ```align_score()``` | Sequence of Maps |
| Persistence | Autocorrelation coeff. of first differences <br> of objects' subsequent map positions | ```persistence_score()``` | Sequence of Maps |



In [ ]:
from evomap.metrics import avg_hitrate_score, misalign_score, persistence_score

In [ ]:
df_eval = pd.DataFrame({
    'Avg Hitrate': avg_hitrate_score(Ys,Ds, input_type = 'distance'),
    'Misalignment': misalign_score(Ys),
    'Persistence': persistence_score(Ys)
}, index = ['EvoMap'])

df_eval

To benchmark these values, we apply t-SNE independently to each distance matrix.

`evomap.mapping` also includes the respective static variant for each method:

In [ ]:
from evomap.mapping import TSNE
tsne_model = TSNE(init = Y_init)

Ys_indep = []
for t in range(n_periods):
    tsne_results = tsne_model.fit_transform(Ds[t])
    Ys_indep.append(tsne_results)

In [ ]:
df_eval = pd.concat((df_eval, pd.DataFrame({
    'Avg Hitrate': avg_hitrate_score(Ys_indep, Ds, input_type = 'distance'),
    'Misalignment': misalign_score(Ys_indep),
    'Persistence': persistence_score(Ys_indep)
}, index = ['Indep. TSNE'])), axis = 0)
df_eval

In [ ]:
df_eval.T.plot(kind = 'bar')

<a id = 'section_5'></a>
## Hyperparameter Selection: 

The metrics introduced in the last section are well suited to tune EvoMap's hyperparameters. 

Specifically, there are two hyperparameters one needs to set when applying EvoMap: 

- alpha (float): Controls the degree of alignment 
- p (int): Controls the degree of smoothing

'Good' values for these parameters naturally depend on the given input data, one should always test multiple values for them and compare their results visually and quantitatively (e.g., using the metrics introduced in the previous section). 

To make such comparisons as easy as possible, EvoMap features a  ```grid_search()``` function. Given some input data and a grid of parameter values, this function creates a map sequence for each parameter combination and summarizes the results. 

First, define the parameter grid as a dictionary. For a start, its always good to define a broad range (which can later be narrowed down more closely): 

In [ ]:
param_grid = {
    'alpha': [0.00001, 0.0001, 0.001], 
    'p': [1,2]}

Next, define which metrics should be used to evaluate each combination from the grid:

In [ ]:
metrics = [misalign_score, persistence_score, avg_hitrate_score]
metric_labels = ['Misalignment', 'Persistence', 'Hitrate']

Then, initialize the model and start the grid search (available for each implementation of EvoMap):

In [ ]:
model = EvoTSNE(verbose = 2, init = Y_inits)

df_grid_results = model.grid_search(
    Xs = Ds, 
    param_grid = param_grid, 
    eval_functions =  metrics,
    eval_labels = metric_labels)

In [ ]:
df_grid_results

In [ ]:
fig, ax = plt.subplots(1,3, figsize = (20,7))

sns.barplot(x = 'alpha', y = 'cost_static_avg', hue = 'p', data = df_grid_results, ax = ax[0])
sns.barplot(x = 'alpha', y = 'Misalignment', hue = 'p', data = df_grid_results, ax = ax[1])
sns.barplot(x = 'alpha', y = 'Persistence', hue = 'p', data = df_grid_results, ax = ax[2])

Based on this evaluation, one can select suitable parameter combinations and inspect them further visually. 

Here, alpha = 0.001 and p = 2 seem reasonable, as this combination decreases misalignment, increases persistence, but does not increase static cost substantially. To use them, either create a new model instance or use the ```set_params()``` function to override the parameters of an existing model instance.

In [ ]:
Y_t = model.set_params({'alpha': 0.001, 'p': 2}).fit_transform(Ds)

In [ ]:
draw_dynamic_map(Y_t, c = sic_codes)

In [ ]:
focus_firms = [
    'INTUIT INC',
    'WESTERN DIGITAL CORP', 
    'APPLE INC']

draw_trajectories(Y_t, labels = names, selected_labels = focus_firms, 
                  title_str = "Selected Trajectories (after tuning)", period_labels = years)

As with any unsupervised learning technique, tuning these maps is both science and art. As there typically is no ground-truth known, it is typically impossible to objectively identify a single best solution. Instead, the grid results should serve as a basis to identify suitable values, from which you should always compare different solutions.    

Further note that the grid was set relatively small to make this tutorial computationally inexpensive. In practical applications, it's always advisable to test a more extensive grid to faithfully identify suitable hyperparameter values. 

<a id = 'section_6'></a>

## Choosing a Different Mapping Method

Thus far, this tutorial only used a single mapping method (t-SNE). While t-SNE works particularly well for large datasets, it has certain properties that make it less suitable for smaller datasets (e.g., map positions are estimated based on nearest neighborhood probabilities, rather than the actual input distances). 

In such (and other) cases, Multidimensional Scaling (MDS) and its variants might be the preferred choice. Therefore, we also provide an implementation of EvoMap for MDS: ```EvoMDS()```

For this demonstration, let's first pick a smaller sample:

In [ ]:
sample_firms = [
    'US CELLULAR CORP', 
    'AT&T INC', 
    'MICROSOFT CORP', 
    'WESTERN DIGITAL CORP', 
    'ORACLE CORP', 
    'EBAY INC', 
    'INTUIT INC', 
    'APPLE INC'] 

df_sample = df_sample.query('name1 in @sample_firms').query('name2 in @sample_firms')

Ds, names, years = build_distance_matrices(df_sample)
n_samples = Ds[0].shape[0]
n_periods = len(Ds)

Note that MDS does not transform the input distances in any way, but rather tries to fit map distances to them as closely as possible. Therefore, always make sure that your input distances are on a reasonable scale. If input distances are very large, for instance, gradient norms can quickly explode and it can become challenging to reach a good solution. Normalizing the input distances to a smaller range can help to avoid such cases and does not affect the resultant maps up to scaling their coordinate system. Make sure, however, to normalize each distance matrix in the sequence by the same factor!

In [ ]:
from evomap.preprocessing import normalize_dist_mats
Ds_norm = normalize_dist_mats(Ds)

Running EvoMap for MDS follows the same syntax as running it for t-SNE. Note, however, that some arguments for both classes differ, as they use different optimization routines. 

**Note:** MDS optimizes a different static cost function than t-SNE (Stress, rather than KL Divergence). As the output of these functions are scaled differently, 'good' values for the hyperparamters can (and often will) differ. Thus, make sure to run the grid search for MDS separately.  

In [ ]:
from evomap.mapping import EvoMDS

param_grid = {
    'alpha': [0.1,1,10], 
    'p': [1,2]}

model_MDS = EvoMDS()

df_res = model_MDS.grid_search(
    Xs = Ds_norm, 
    param_grid = param_grid, 
    eval_functions =  metrics,
    eval_labels = metric_labels)

For (very) high values of alpha, lower step sizes are required to ensure convergence. Otherwise, the very large temporal gradient can explode and the optimization diverges. EvoMap implements a set of controls trying to avoid such behavior. For instance, it automatically adjusts (i.e., lowers) step sizes if the gradient norm starts to diverge. In most cases, these controls will suffice to find good solutions. If not, consider lowering alpha or decrease step sizes manually. 

In [ ]:
fig, ax = plt.subplots(1,3, figsize = (20,7))

sns.barplot(x = 'alpha', y = 'cost_static_avg', hue = 'p', data = df_res, ax = ax[0])
sns.barplot(x = 'alpha', y = 'Misalignment', hue = 'p', data = df_res, ax = ax[1])
sns.barplot(x = 'alpha', y = 'Persistence', hue = 'p', data = df_res, ax = ax[2])

From this graph, alpha = 1 and p = 2 seem reasonable. For higher values of alpha, static cost rises substantially. 

In [ ]:
model_MDS.set_params({'alpha': 1, 'p': 2})
Ys_MDS = model_MDS.fit_transform(Ds_norm)

In [ ]:
draw_dynamic_map(Ys_MDS, show_arrows = True)

In [ ]:
draw_trajectories(Ys_MDS, labels = names, title_str = "Selected Trajectories (MDS; after tuning)", period_labels = years)

<a id = 'section_7'></a>
## Special Cases, Extensions and Troubleshooting

**WORK IN PROGRESS*